# Analysis - Standard
## Streamflow Benchmarking Tutorial

<img src='../../../doc/assets/Eval_Analysis.svg' width=600>

## Essential Benchmark Components
This benchmark notebook will present a workflow which follows a canonical model for Essential Benchmark Components: 
1) A set of predictions and matching observation (the data); 
2) The domain (e.g. space or time) over which to benchmark;
3) A set of statistical metrics with which to benchmark. 

## What is the "Standard Suite"?
The 'Standard Suite' is a group of statistical metrics used for benchmarking streamflow from hydrologic models and comes from [Towler et al., 2023](https://doi.org/10.5194/hess-27-1809-2023). These metrics were chosen in order to examine multiple aspects of hydrologic model simulation of streamflow. Though, it is important to note that these metrics are applicable to other water budget components (similar to [d-score](02_Analysis_DScore.ipynb)). Let's calculate these series of statistical metrics between our simulated and observed data in this notebook, and then explore the results in the next.

#### Let's get started.

## Step 0: Load libraries and configure Python computing environment

In [ ]:
import pandas as pd
import logging
import os

## Step 1: Load Data
<img src='../../../doc/assets/Eval_Analysis_Data.svg' width=360>

### Essential Benchmark Components: 
1) **A set of predictions and matching observations** <span style="color:red; font-size:large"><<--You are here</span>
2) The domain over which to benchmark 
3) A set of statistical metrics with which to benchmark.

Finding and loading data is made easier for this particular notebook example (using the _streamflow_ variable), in that most of the required data has been pre-processed and stored in a cloud-friendly format. That process is described in the **[data preparation](01_Data_Prep.ipynb)**
notebook. We will proceed here using the pre-prepared data for _streamflow_, which is included in the HyTEST **[intake catalog](../../../dataset_catalog/README.md)**.

> Learn more about `intake` [here](../../../dataset_catalog/README.md)

In [ ]:
import intake
cat = intake.open_catalog(r'https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml')
print("Available datasets: \n", "\n".join(cat.keys()))

The above list represents the processed datasets available for benchmarking.  If a dataset
you want is not in that list, 
you can load data directly from S3 or 'onprem' (on-premises). 
If you load data from a source other than this list, you can jump to **Step 2: Restrict to a Domain**

Note that some of the datasets in the cataloged list of data above are duplicated: Some are `-onprem` , some are `-cloud`, and some are `-osn`. This means it is the same data, but the storage location and access protocol will be different. 
You will definitely want to use the correct copy of the data for your computing environment. The idea is to "bring the compute to the data".
* `onprem` : Direct access via the USGS internal `caldera` filesystem from _denali_ or _tallgrass_, and now on _hovenweep_.
* `osn` : Network access via OSN pod, which uses the S3 API, suitable for consumption on any jupyter server (onprem, cloud, or local)
* `cloud` : Network access via S3 bucket, suitable for consumption on cloud-hosted Jupyter servers. You could also access using any network-attached computer, but the amount of data will likely saturate your connection.  Use in the cloud is okay (e.g. ESIP Nebari or HyTEST Nebari).

So... are you on-prem? If so, see and uncomment lines below. If not, don't worry the default is set up to pull data from the OSN. 
>Note: it is possible to still use datasets hosted on OSN when computing on the supercomputers, though we try to always 'bring the compute to the data'.

In [ ]:
## No, I am on cloud or working locally
## Then you'll want to use the following
obs_data_src='nwis-streamflow-usgs-gages-osn'
mod_data_src='nwm21-streamflow-usgs-gages-osn'
print("Observed : ", obs_data_src)
print("Modeled  : ", mod_data_src)

In [ ]:
## Yes I am on the supercomputer! 
## Then you'll want to uncomment and use the below datasets:
#obs_data_src='nwis-streamflow-usgs-gages-onprem'
#mod_data_src='nwm21-streamflow-usgs-gages-onprem'
#print("Observed : ", obs_data_src)
#print("Modeled  : ", mod_data_src)

In [ ]:
variable_of_interest = 'streamflow'
try:
    obs = cat[obs_data_src].to_dask()
    mod = cat[mod_data_src].to_dask()
except KeyError:
    print("Something wrong with dataset names.")
    raise

try:
    obs_data = obs[variable_of_interest]
    mod_data = mod[variable_of_interest].astype('float32')
except KeyError:
    print(f"{variable_of_interest} was not found in these data.")

obs_data.name = 'observed'
mod_data.name = 'simulated'    

## Step 2: Restrict to a Domain

<img src='../../../doc/assets/Eval_Analysis_Domain.svg' width=360>

Essential Benchmark Components: 
1) A set of predictions and matching observations,  
2) **The domain over which to benchmark** <span style="color:red; font-size:large"><<--You are here</span>
3) A set of statistical metrics with which to benchmark. 

Each benchmark domain is defined over specific bounds (typically space and/or time). 
Benchmark domain definitions can be defined within the notebook, or sourced from
elsewhere. For this example, we use the _Cobalt_ gages, avaliable for download on ScienceBase 
([Foks et al., 2022](https://doi.org/10.5066/P972P42Z)).  

Each benchmark domain is defined over specific bounds (typically space and/or time). 
Benchmark domain definitions can be defined within the notebook, or sourced from
elsewhere. For this example, we use the _Cobalt_ gages, avaliable for download on ScienceBase 
([Foks et al., 2022](https://doi.org/10.5066/P972P42Z)). This is essentially a list of streamflow gages of interest to HyTEST, along with some metadata about that gage (watershed, 
reachcode, etc).  We will use this as a spatial selector to restrict the observed and modeled/simulated data to only the gages found within this benchmarking domain.

In [ ]:
cobalt = pd.read_csv(
    'https://www.sciencebase.gov/catalog/file/get/6181ac65d34e9f2789e44897?f=__disk__22%2F1a%2Fd2%2F221ad2fe9d95de17731ad35d0fc6b417a4b53ee1',
    dtype={'site_no':str, 'huc_cd':str, 'reachcode':str, 'comid':str, 'gagesII_class':str, 'aggecoregion': str}, 
    index_col='site_no'
    )
# Re-format the gage_id/site_no string value.  ex:   "1000000"  ==> "USGS-1000000"
cobalt.rename(index=lambda x: f'USGS-{x}', inplace=True)
print(f"{len(cobalt.index)} gages in this benchmark")
cobalt.head(2)

So now we have a domain dataset representing the stream gages (unique `site_no` values) identifying the locations making up the spatial domain of this benchmark. While we have good metadata for these gages (lat/lon location, HUC8 code, etc), we will only use the list of `site_no` values to select locations out of the raw data.

## Step 3: Define Metrics

<img src='../../../doc/assets/Eval_Analysis_Metrics.svg' width=360>

Essential Benchmark Components: 
1) A set of predictions and matching observations  
2) The domain over which to benchmark 
3) **A set of statistical metrics with which to benchmark** <span style="color:red; font-size:large"><<--You are here</span>

The code to calculate the various NWM metrics has been standardized in
{doc}`/evaluation/Metrics_StdSuite_v1`
with usage examples in 
{doc}`/evaluation/tutorials/stats/Usage_StdSuite_v1`. 
You can use these metrics or write your own.  To import and use these standardized definitions, run this cell:

In [ ]:
%run ../../Metrics_StdSuite_v1.ipynb

Whether you use these functions or your own, we need to put all metric computation into a special all-encompasing 
benchmarking function--a single call which can be assigned to each gage in our domain list. This sort of arrangement 
is well-suited to parallelism with `dask`. 

If this is done well, the process will benefit enormously from task parallelism -- each gage can be given its own 
CPU worker to run on.  After all are done, the various results will be collected and assembled into a composite dataset. 

To achieve this, we need a single 'atomic' function that can execute independently. It will take the gage identifier 
as input and return a list of metrics.

In [ ]:
## Wrapper function -- this func will be called once per gage_id, each call on its own dask worker
def compute_benchmark(gage_id):
    try:
        ## obs_data and mod_data should be globals...
        obs = obs_data.sel(gage_id=gage_id).load(scheduler='single-threaded').to_series()
        mod = mod_data.sel(gage_id=gage_id).load(scheduler='single-threaded').to_series().resample('1D', offset='5h').mean() # note the time shift for this
    
        # make sure the indices match
        obs.index = obs.index.to_period('D')
        mod.index = mod.index.to_period('D')

        # merge obs and predictions; drop NaNs.
        gage_df = pd.merge(obs, mod, left_index=True, right_index=True).dropna(how='any')
        
        scores = pd.Series(
            data={
                'NSE': NSE(gage_df.observed, gage_df.simulated),
                'KGE': KGE(gage_df.observed, gage_df.simulated),
                'logNSE': logNSE(gage_df.observed, gage_df.simulated),
                'pbias': pbias(gage_df.observed, gage_df.simulated),
                'rSD': rSD(gage_df.observed, gage_df.simulated),
                'pearson': pearson_r(gage_df.observed, gage_df.simulated),
                'spearman': spearman_r(gage_df.observed, gage_df.simulated), 
                'pBiasFMS': pBiasFMS(gage_df.observed, gage_df.simulated),
                'pBiasFLV': pBiasFLV(gage_df.observed, gage_df.simulated),
                'pBiasFHV': pBiasFHV(gage_df.observed, gage_df.simulated)
            },
            name=gage_id,
            dtype='float64'
        )
        return scores
    except Exception as e:#<-- this is an extremely broad way to catch exceptions.  We only do it this way to ensure 
                          #    that a failure on one benchmark (for a single stream gage) will not halt the entire run. 
        logging.info("Benchmark failed for %s", gage_id)
        return None

Let's test to be sure this `compute_benchmark()` function will return data for a single gage

In [ ]:
compute_benchmark('USGS-01030350')

## Execute the Analysis 
We will be doing a lot of work in parallel, using workers within a 'cluster'.  
The details of cluster configuration are handled for us by 'helper' notebooks, below. 
You can override their function by doing your own cluster configuration if you like. 

In [ ]:
# uncomment the lines below to read in your AWS credentials if you want to access data from a requester-pays bucket (-cloud)
# os.environ['AWS_PROFILE'] = 'default'
# %run ../../../environment_set_up/Help_AWS_Credentials.ipynb

In [ ]:
%run ../../../environment_set_up/Start_Dask_Cluster_Nebari.ipynb
### Executes external 'helper to spin up a cluster of workers. 

We verified above that the `compute_benchmark` works on the "hosted" server (where this
notebook is being executed. As a sanity check before we give the cluster of workers a lot 
to do, let's verify that we can have a remote worker process a gage by submitting work
to one in isolation: 

In [ ]:
client.submit(compute_benchmark, 'USGS-01030350').result()

Now that we've got a benchmark function, and can prove that it works in remote workers 
within the cluster, we can dispatch a fleet of workers to process our data in parallel.
We will make use of `dask` to do this using a dask '_bag_'.  

>Read more about task parallelism with Dask and how we are using dask bags [here](../../../essential_reading/Parallel_Dask.ipynb)

In [ ]:
# Set up a dask bag with the contents being a list of the cobalt gages
import dask.bag as db

# For the first 100 gages within the Cobalt list, try to map the compute benchmark function over them.
# Remove '[0:100]' to run full list (may take awhile depending on cores/workers available)
bag = db.from_sequence(cobalt.index.tolist()[0:100]).map(compute_benchmark)

results = bag.compute() #<< Dispatch the workers

In [ ]:
# preview
print("Number of gages:", len(results))
results[0:2]

With that big task done, we don't need `dask` parallelism any more. Let's shut down the cluster. This is important to in order to free up workers for other users on cloud in particular.

In [ ]:
client.close(); del client
cluster.close(); del cluster

## Assemble the results
The `bag` now contains a collection of return values (one per call to `compute_benchmark()`).  We can massage that into a table/dataframe for easier processing: 

In [ ]:
r = [i for i in results if i is not None] # Drop entries where compute_benchmark failed
results_df = pd.concat(r, axis=1).T
results_df.index.name = 'site_no'
results_df

This dataframe/table can be saved to disk as a CSV. It will be used for visualizations in [other notebooks](03_Vizualization.ipynb).

In [ ]:
results_df.to_csv('Standard_streamflow_example.csv') ##<--- change this to a personalized filename if desired.